### Imports

In [ ]:
import cv2
import torch
import numpy as np
import pandas as pd
from PIL import Image, ImageEnhance, ImageFilter
from PIL.ImageOps import invert

In [ ]:
show_intermediate = False   # set to True to show intermediate results
output = []                 # create list to store all final outputs

### Image preprocessing

In [ ]:
test_image = '../assets/example_image.jpg'

# convert to grayscale
img = Image.open(test_image).convert('L')
img = invert(img)
img = ImageEnhance.Contrast(img).enhance(2)
img = img.point(lambda p: p > 220 and 255)
img = img.filter(ImageFilter.SMOOTH)

# show image
if show_intermediate:
    img.show()

### Components Inference

In [ ]:
# Get the model
model_path = '../models/components.pt' 
c_model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)
c_model.eval()

In [ ]:
# Inference
c_results = c_model(img)

# Create a new dataframe with the results
for result in c_results.xyxy:
    output.append([result[5], result[0], result[1], result[2], result[3], result[4]])

# Print and show results
if show_intermediate:
    print(c_results.pandas().xyxy)
    c_results.show()

### Junction inference

In [ ]:
model_path = '../models/junctions.pt'
j_model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)
j_model.eval()

In [ ]:
# Perform inference on the image without components
j_results = j_model(img)

# Print and show results
if show_intermediate:
    print(j_results.pandas().xyxy)
    j_results.show()

### Post processing

In [ ]:
# Remove overlapping junctions (TODO: move this function to utils later)
def remove_overlapping_junctions(j_results, c_results, overlap_threshold=0.1):
    """ Remove junctions that are overlapping with components

    Args:
        j_results (yolov5.results): Results of the junction detection model
        c_results (yolov5.results): Results of the component detection model

    Returns:
        list: List of coordinates of components with confidence scores and class labels
        list: List of coordinates of junctions that are not overlapping with components with confidence scores and class labels
    """
    # Create list to store junctions to be removed
    j_to_remove = []

    # Get the bounding boxes of the junctions and the components
    j_boxes = j_results.xyxy[0]
    c_boxes = c_results.xyxy[0]

    # Get the coordinates of the junctions and the components
    j_coords = [(int(box[0]), int(box[1]), int(box[2]), int(box[3]), np.round(float(box[4]), 4), int(box[5])) for box in j_boxes]
    c_coords = [(int(box[0]), int(box[1]), int(box[2]), int(box[3]), np.round(float(box[4]), 4), int(box[5])) for box in c_boxes]

    # Remove junctions that are overlapping with components
    for c_coord in c_coords:
        for j_coord in j_coords:
            # Calculate percentage of junction that is overlapping with component
            x1 = max(c_coord[0], j_coord[0])
            y1 = max(c_coord[1], j_coord[1])
            x2 = min(c_coord[2], j_coord[2])
            y2 = min(c_coord[3], j_coord[3])

            # Calculate area of intersection
            intersection = max(0, x2 - x1 + 1) * max(0, y2 - y1 + 1)

            # Calculate area of junction
            j_area = (j_coord[2] - j_coord[0] + 1) * (j_coord[3] - j_coord[1] + 1)

            # Calculate percentage of junction that is overlapping with component
            overlap = intersection / j_area

            # Remove junction if it is overlapping with component
            if overlap > overlap_threshold:
                j_to_remove.append(j_coord)
    
    # Remove junctions that are overlapping with components
    j_coords = [j_coord for j_coord in j_coords if j_coord not in j_to_remove]

    return c_coords, j_coords

# Perform non-maximum suppression (TODO: move this function to utils later)
def non_max_suppression_fast(j_coords, iou_threshold=0.5):
    """ Apply non-maximum suppression to the coordinates of the junctions
    Args:
        j_coords (list): List of coordinates of junctions with confidence scores and class labels
        iou_threshold (float): Intersection over Union (IoU) threshold
    Returns:
        list: List of coordinates of junctions after non-maximum suppression with confidence scores and class labels
    """
    if len(j_coords) == 0:
        return []
    
    # Transform the coordinates into a numpy array
    boxes = np.array([[x1, y1, x2, y2] for x1, y1, x2, y2, score, label in j_coords])
    scores = np.array([score for x1, y1, x2, y2, score, label in j_coords])

    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    # Compute the area of the bounding boxes
    area = (x2 - x1 + 1) * (y2 - y1 + 1)

    # Sort by confidence
    indices = np.argsort(scores)[::-1]

    keep = []

    # Iterate over the bounding boxes
    while len(indices) > 0:
        current = indices[0]
        keep.append(current)

        xx1 = np.maximum(x1[current], x1[indices[1:]])
        yy1 = np.maximum(y1[current], y1[indices[1:]])
        xx2 = np.minimum(x2[current], x2[indices[1:]])
        yy2 = np.minimum(y2[current], y2[indices[1:]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        overlap = (w * h) / area[indices[1:]]

        suppressed_indices = np.where(overlap <= iou_threshold)[0]

        indices = indices[suppressed_indices + 1]

    return [j_coords[i] for i in keep]


# Remove overlapping junctions
c_coords, j_coords = remove_overlapping_junctions(j_results, c_results)

# Remember the old junctions
columns = ['xmin', 'ymin', 'xmax', 'ymax', 'confidence', 'class']
old_j_df = pd.DataFrame(j_coords, columns=columns)

# Perform non-maximum suppression on junctions
j_coords = non_max_suppression_fast(j_coords)

# Create dataframes from the components and remaining junctions
c_df = pd.DataFrame(c_coords, columns=columns)
j_df = pd.DataFrame(j_coords, columns=columns)

if show_intermediate:
    print("Components:")
    print(c_df)

    print("Old junctions:")
    print(old_j_df)

    print("NMS Junctions:")
    print(j_df)

### Show final detections

In [ ]:
# using c_df containing the components and j_df containing the junctions, draw all bounding boxes on the original image
img = cv2.imread(test_image)

# Draw components
for index, row in c_df.iterrows():
    xmin = int(row['xmin'])
    ymin = int(row['ymin'])
    xmax = int(row['xmax'])
    ymax = int(row['ymax'])
    cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

# Draw junctions
for index, row in j_df.iterrows():
    xmin = int(row['xmin'])
    ymin = int(row['ymin'])
    xmax = int(row['xmax'])
    ymax = int(row['ymax'])
    cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)

# Save the image
cv2.imwrite('../assets/output.jpg', img)

### Convert to generated image
Take the final output list and generate the digital circuit based on that

In [ ]:
# TODO